Detección de caras con webcam

In [4]:
import cv2
import time
import FaceNormalizationUtils as faceutils
# My face detectors interface
import FaceDetectors
import numpy as np
import math
import dlib


En el bucle de procesamiento, las teclas 'd' y 'e' `permiten respectivamenet cambiar de modelo de detección de caras, y en su caso de máscara de detección del rostro.

La ejecución de la siguiente celda produce error al no disponer de los archivos shape_predictor_5_face_landmarks.dat y shape_predictor_68_face_landmarks.dat que por su tamaño no se incluyeron en el repositorio. Pueden descargarse desde el enlace proporcionado en el campus virtual (opción aconsejada), o
desde el [repositorio de archivos de dlib](http://dlib.net/files/).



In [20]:
# Inicializar el detector de caras y puntos faciales
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

# Inicializar la cámara
cap = cv2.VideoCapture(0)

# Tamaño del píxel para pixelar la cara
pixel_size = 10

# Variable para realizar o desactivar la censura
censor_mode = 0  # 0: Sin censura, 1: Censura de ojos, 2: Pixelación de cara, 3: Censura gaussiana

def calculate_angle(A, B, C):
    """Calcula el ángulo entre tres puntos A, B y C. B es el vértice del ángulo."""
    BA = (A[0] - B[0], A[1] - B[1])
    BC = (C[0] - B[0], C[1] - B[1])
    cosine_angle = (BA[0] * BC[0] + BA[1] * BC[1]) / (math.sqrt(BA[0]**2 + BA[1]**2) * math.sqrt(BC[0]**2 + BC[1]**2))
    angle = math.acos(cosine_angle)
    return math.degrees(angle)

def mouth_aspect_ratio(mouth):
    angle = calculate_angle(mouth[0], mouth[6], mouth[9])  # Puntos 48, 54 y 57
    ANGLE_THRESHOLD = 35  # Ajusta este valor según sea necesario
    print("Angulo= " + str(angle))
    return angle > ANGLE_THRESHOLD

def detect_smile(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)
    
    smile_detected = False

    for face in faces:
        landmarks = predictor(gray, face)
        
        # Extracción de puntos de la boca
        mouth_points = []
        for n in range(48, 68):
            x = landmarks.part(n).x
            y = landmarks.part(n).y
            mouth_points.append((x, y))
            cv2.circle(image, (x, y), 2, (0, 255, 0), -1)
        # Detección de la sonrisa
        if mouth_points:
            smile_detected = mouth_aspect_ratio(mouth_points)
            print("Sonriendo = " + str(smile_detected))
        

    return smile_detected, image

while True:
    ret, frame = cap.read()

    # Detectar caras en el marco
    faces = detector(frame)

    for face in faces:
        # Obtener landmarks faciales
        shape = predictor(frame, face)

        if censor_mode == 1:
            margen_horizontal = 10  # Margen adicional en cada lado horizontalmente
            margen_vertical = 5     # Margen adicional en cada lado verticalmente

            left_eye_x = min(shape.part(36).x, shape.part(39).x) - margen_horizontal
            right_eye_x = max(shape.part(42).x, shape.part(45).x) + margen_horizontal
            top_eye_y = min(shape.part(37).y, shape.part(38).y, shape.part(43).y, shape.part(44).y) - margen_vertical
            bottom_eye_y = max(shape.part(41).y, shape.part(40).y, shape.part(47).y, shape.part(46).y) + margen_vertical

            # Asegurarse de que las coordenadas no sean menores que 0 o mayores que las dimensiones de la imagen
            left_eye_x = max(left_eye_x, 0)
            right_eye_x = min(right_eye_x, frame.shape[1])
            top_eye_y = max(top_eye_y, 0)
            bottom_eye_y = min(bottom_eye_y, frame.shape[0])

            # Dibujar el rectángulo relleno
            frame[top_eye_y:bottom_eye_y, left_eye_x:right_eye_x] = (0, 0, 0)
        elif censor_mode == 2:
            # Pixelar la cara
            x, y, w, h = face.left(), face.top(), face.width(), face.height()
            face_roi = frame[y:y + h, x:x + w]
            face_roi = cv2.resize(face_roi, (w // pixel_size, h // pixel_size))
            face_roi = cv2.resize(face_roi, (w, h), interpolation=cv2.INTER_LINEAR)
            frame[y:y + h, x:x + w] = face_roi
        elif censor_mode == 3:
            # Aplicar censura gaussiana
            x, y, w, h = face.left(), face.top(), face.width(), face.height()
            face_roi = frame[y:y + h, x:x + w]
            face_roi = cv2.GaussianBlur(face_roi, (0, 0), sigmaX=30)
            frame[y:y + h, x:x + w] = face_roi
        elif censor_mode == 4:
            is_smiling, processed_frame = detect_smile(frame)
            cv2.imshow("Smile Detector", processed_frame)
            #print("Smiling:", is_smiling)  # Imprimir el estado de la sonrisa

    # Mostrar el marco con la censura correspondiente
    cv2.imshow('Censor Mode', frame)

    key = cv2.waitKey(1)

    if key == 27:  # Presiona Esc para salir
        break
    elif key == ord('d'):  # Presiona "d" para alternar entre los modos de censura
        censor_mode = (censor_mode + 1) % 5
    elif key == ord('e'):  # Presiona "d" para alternar entre los modos de censura
        censor_mode = (censor_mode - 1) % 5

cap.release()
cv2.destroyAllWindows()

Angulo= 32.47119229084849
Sonriendo = False
Angulo= 31.652736829159863
Sonriendo = False
Angulo= 32.005383208083494
Sonriendo = False
Angulo= 32.45643252314632
Sonriendo = False
Angulo= 32.005383208083494
Sonriendo = False
Angulo= 31.218402764346386
Sonriendo = False
Angulo= 30.699722550814407
Sonriendo = False
Angulo= 30.699722550814407
Sonriendo = False
Angulo= 32.005383208083494
Sonriendo = False
Angulo= 32.44457249857691
Sonriendo = False
Angulo= 32.005383208083494
Sonriendo = False
Angulo= 32.005383208083494
Sonriendo = False
Angulo= 32.45643252314632
Sonriendo = False
Angulo= 32.828541791412526
Sonriendo = False
Angulo= 32.828541791412526
Sonriendo = False
Angulo= 32.828541791412526
Sonriendo = False
Angulo= 34.11447294534127
Sonriendo = False
Angulo= 30.699722550814407
Sonriendo = False
Angulo= 33.231711067979354
Sonriendo = False
Angulo= 34.54516492224651
Sonriendo = True
Angulo= 31.827446576673108
Sonriendo = False
Angulo= 32.34744349944203
Sonriendo = False
Angulo= 33.6900675

In [18]:
normalizatorHS = faceutils.Normalization()

# Face detectors interface
FDet = FaceDetectors.FaceDetector()

# Fonts
font = cv2.FONT_HERSHEY_SIMPLEX

# Webcam connection
cap = cv2.VideoCapture(0)
# Check for other cameras
if not cap.isOpened():
    cap = cv2.VideoCapture(1)
    if not cap.isOpened():
        cap = cv2.VideoCapture(0)
        if not cap.isOpened():
            print('Camera error')
            exit(0)
        else:
            print('Camera 0')
    else:
        print('Camera 1')
else:
    print('Camera 0')

    # Face detection and eye model setup
imodoF = 0
imodoE = 0

debug = 0

#Set camera resolution
cap.set(3,640);
cap.set(4,480);

while True:
    # Get frame
    t = time.time()
    ret, frame = cap.read()
    # For HS normalization
    B, G, R = cv2.split(frame)

    # Search face with a specific setup for face and eye detection
    values = FDet.SingleFaceEyesDetection(frame, FDet.FaceDetectors[imodoF], FDet.EyeDetectors[imodoE])
    if values is not None:
        face, eyes, shape = values

        #draws face container
        [x, y , w, h] = face
        if x > -1:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

            # draws eyes and mask if available
            [lex, ley, rex, rey] = eyes
            if lex > -1:
                # Show detected facial elements
                if imodoF > 0:
                    for (x, y) in shape:
                        cv2.circle(frame, (x, y), 2, (255, 255, 255), -1)

                cv2.circle(frame, ((int)(lex), (int)(ley)), 4, (0, 0, 255), -1)
                cv2.circle(frame, ((int)(rex), (int)(rey)), 4, (0, 255, 0), -1)


                # Normalize and show
                # color channels
                normalizatorHS.normalize_gray_img(B, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                Bnorm = normalizatorHS.normf_image
                normalizatorHS.normalize_gray_img(G, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                Gnorm = normalizatorHS.normf_image
                normalizatorHS.normalize_gray_img(R, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                Rnorm = normalizatorHS.normf_image
                NormBGR = cv2.merge((Bnorm, Gnorm, Rnorm))
                cv2.imshow("Normalized", NormBGR)


    if debug:
        print("Processing time : {:.3f}".format(time.time() - t))

    # Show resulting image
    cv2.putText(frame, FDet.FaceDetectors[imodoF], (10, 20), font, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
    if imodoF == 1 or imodoF == 2:
        cv2.putText(frame, FDet.EyeDetectors[imodoE], (50, 20), font, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.imshow('Cam', frame)
    
    # Esc to finish
    tec = cv2.waitKey(40)
    if tec & tec == 27:  # Esc
        break
    # Face detector change
    elif tec & 0xFF == ord('d'):
        imodoF = imodoF + 1
        if imodoF >= len(FDet.FaceDetectors):
            imodoF = 0
    #Eye detector change
    elif tec & 0xFF == ord('e'):
        imodoE = imodoE + 1
        if imodoE >= len(FDet.EyeDetectors):
            imodoE = 0

# Close windoews and release camera
cap.release()
cv2.destroyAllWindows()


Camera 0
